In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('mode.chained_assignment',  None)

merged_data = pd.read_csv('./data/final_merge_bus_data.csv')

# getrouteinfoall
getrouteinfoall = pd.read_csv('./data/getRouteInfoAll.csv', encoding = 'cp949', index_col = 0)
getrouteinfoall_df = getrouteinfoall[['ROUTE_CD','ROUTE_NO']]

### gestationbyrouteall
getstationbyrouteall = pd.read_csv('./data/getStationByRouteAll.csv', encoding = 'cp949', index_col = 0)
getstationbyrouteall_df1 = getstationbyrouteall[['ROUTE_CD','BUSSTOP_SEQ','BUSSTOP_TP','GPS_LATI','GPS_LONG']]
getstationbyrouteall_df2 = getstationbyrouteall[['ROUTE_CD','BUS_STOP_ID','BUSSTOP_SEQ','BUSSTOP_TP','GPS_LATI','GPS_LONG']]

In [2]:
# 문자열 제거
merged_data['YYYY-MM-DD'] = merged_data['YYYY-MM-DD'].str[:10]
# 'YYYY-MM-DD' 열을 datetime 형식으로 변환
merged_data['YYYY-MM-DD'] = pd.to_datetime(merged_data['YYYY-MM-DD'], format='%Y-%m-%d')
# DAY 변수 추가
merged_data['DAY'] = merged_data['YYYY-MM-DD'].dt.day

merged_data_copy = merged_data.iloc[:,2:]

---

In [3]:
def bus_average_data(exdata, route_no): # 노선별 평균값 데이터 생성 함수
    day_values = [1, 2, 5, 7, 8, 9, 12, 13, 14, 15, 16, 20, 21, 22, 23, 26, 27, 28, 29, 30]

    average_data = pd.DataFrame()

    for day in day_values:
        bus_data = exdata[(exdata['노선'] == route_no) & (exdata['DAY'] == day)].copy() # 특정 노선과 요일에 해당하는 데이터 필터링
        bus_data.drop(labels=['노선', '정류장', 'DAY'], axis=1, inplace=True)
        bus_data = bus_data.set_index('정류장순번') # 정류장순번을 index로 설정
        average_data = pd.concat([average_data, bus_data])

    average_data = average_data.groupby('정류장순번').mean() # 선택한 요일들에 대한 각 정류장의 평균값 계산
    
    return average_data

def get_bus_stop_tp(getstationbyrouteall_df, ROUTE_BUS_NUMBER, ROUTE_NO):
    # 주어진 버스 노선 번호에 해당하는 버스 노선 코드를 얻은 뒤, 노선 코드에 해당하는 버스 정류장 데이터 추출 <순서대로 진행하지 않으면 버스 정류장 데이터가 중복 됨>
    exdata = getstationbyrouteall_df[getstationbyrouteall_df['ROUTE_CD']==ROUTE_BUS_NUMBER[ROUTE_BUS_NUMBER['ROUTE_NO']==ROUTE_NO]['ROUTE_CD'].values[0]].reset_index(drop=True)
    
    return exdata

def final_data(AVERAGE_DATA, BUS_BUSSTOP_TP):
    exdata = AVERAGE_DATA.merge(BUS_BUSSTOP_TP, left_on = '정류장순번', right_on = 'BUSSTOP_SEQ').set_index('BUSSTOP_SEQ')
    STARTING_POINT = exdata[exdata['BUSSTOP_TP']=='2'].index[0]  # 하행 방향의 첫번째 정류장의 인덱스
    
    # 상행 방향 데이터와 하행 방향 데이터로 분리하여 생성
    exdata_UP = exdata[:STARTING_POINT]
    exdata_UP.drop(labels = ['ROUTE_CD','BUSSTOP_TP'], axis = 1, inplace = True)
    exdata_DOWN = exdata[STARTING_POINT:]
    exdata_DOWN.drop(labels = ['ROUTE_CD','BUSSTOP_TP'], axis = 1, inplace = True)
    
    return exdata_UP, exdata_DOWN # ex) BUS_102_UP, BUS_102_DOWN

##### 히트맵 #####

bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
directions = ['UP', 'DOWN']

# # 히트맵 그리는 함수
# def plot_bus_heatmap(bus_number, direction):
#     plt.rc('font', family='malgun gothic')
#     plt.figure(figsize=(20, 14))
#     heatmap_data = globals().get(f'BUS_{bus_number}_{direction}')
#     heatmap_data = heatmap_data.iloc[:,:19]
#     sns.heatmap(heatmap_data, annot=True, fmt='.1f', linewidths=.5)
#     plt.title(f'{bus_number}번 {direction} 버스 히트맵')
#     plt.xlabel('시간대')
#     plt.ylabel('정류장순번')
    
#     #  # 그래프를 이미지 파일로 저장
#     # image_path = f'./IMAGE_FILE/{bus_number}_{direction}_heatmap.png'
#     # plt.savefig(image_path, bbox_inches='tight')
    
#     plt.show()

# 히트맵 그리는 함수
def plot_bus_heatmap(bus_number, direction):
    plt.rc('font', family='malgun gothic')
    plt.figure(figsize=(20, 20))
    
    if direction == 'UP':
        heatmap_data = globals().get(f'BUS_{bus_number}_UP')
    elif direction == 'DOWN':
        heatmap_data = globals().get(f'BUS_{bus_number}_DOWN')
    
    heatmap_data = heatmap_data.iloc[:, :19]  # 좌표 데이터 제거 후 히트맵 표시
    
    sns.heatmap(heatmap_data, annot=True, fmt='.1f', vmin=0, vmax=100)
    plt.title(f'{bus_number}번 {direction} 버스 히트맵')
    plt.xlabel('시간대')
    plt.ylabel('정류장순번')

    #  # 그래프를 이미지 파일로 저장
    # image_path = f'./IMAGE_FILE/{bus_number}_{direction}_heatmap.png'
    # plt.savefig(image_path, bbox_inches='tight')

    plt.show()

### EX) plot_bus_heatmap(102, 'UP') ###

---

# 히트맵 그리기

In [4]:
# 각 노선별로 평균 데이터프레임 생성하는 코드 ex) AVERAGE_DATA_102 : 102번 버스의 평균값
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
average_data_dict = {}

for bus_number in bus_numbers:
    average_data = bus_average_data(merged_data_copy, bus_number)
    average_data_dict[bus_number] = average_data

for bus_number in bus_numbers:
    df_name = f"AVERAGE_DATA_{bus_number}"
    globals()[df_name] = average_data_dict[bus_number]

In [5]:
# 필요한 노선만 필터링한 뒤, int형으로 변환
bus_numbers = ['102', '103', '105', '106', '108', '115', '119', '201', '211', '301', '311', '314', '511', '603', '604', '605', '613', '615', '703', '704', '705', '706', '802']
ROUTE_BUS_NUMBER = getrouteinfoall_df[getrouteinfoall_df['ROUTE_NO'].isin(bus_numbers)]
ROUTE_BUS_NUMBER['ROUTE_NO'] = ROUTE_BUS_NUMBER['ROUTE_NO'].astype(int)

In [6]:
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802] # 위에서 str타입으로 불러왔으니 다시 불러오기

for bus_number in bus_numbers: # 노선(버스)별로 BUSSTOP_TP를 필터링해줌
    variable_name = f"BUS_{bus_number}_BUSSTOP_TP" # BUS_102_BUSSTOP_TP
    globals()[variable_name] = get_bus_stop_tp(getstationbyrouteall_df1, ROUTE_BUS_NUMBER, bus_number)

In [7]:
# 각 버스별로 상/하행 데이터 생성
for bus_number in bus_numbers:
    average_data = globals().get(f'AVERAGE_DATA_{bus_number}')  # ex: AVERAGE_DATA_102
    busstop_tp_data = globals().get(f'BUS_{bus_number}_BUSSTOP_TP')  # ex: BUS_102_BUSSTOP_TP
    
    bus_number_str = str(bus_number).zfill(3)  # 버스번호를 3자리로 맞춤
    bus_name = f'BUS_{bus_number_str}'
    
    # 상/하행 데이터 생성
    up_data, down_data = final_data(average_data, busstop_tp_data)
    # 데이터 저장
    globals()[f'{bus_name}_UP'] = up_data
    globals()[f'{bus_name}_DOWN'] = down_data

In [8]:
# plot_bus_heatmap(105, 'UP')

In [9]:
# 버스 전체 히트맵 저장
# bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
# up_down = ['UP', 'DOWN']

# def plot_and_save_heatmaps():
#     for bus_number in bus_numbers:
#         for direction in up_down:
#             plot_bus_heatmap(bus_number, direction)
#             print(f'{bus_number}번 {direction} 버스 히트맵 생성 및 저장 완료.')

# # 이미지 파일로 저장
# plot_and_save_heatmaps()

## folium heatmapwithtime

### 출퇴근 시간대 노선
> 105번, 604번, 605번, 703번, 705번

In [10]:
# from sklearn.preprocessing import MinMaxScaler
# minMaxScaler = MinMaxScaler()
# print(minMaxScaler.fit(BUS_102_UP.iloc[:,:19]))
# BUS_102_UP_minMaxScaled = minMaxScaler.transform(BUS_102_UP.iloc[:,:19])
# BUS_102_UP_minMaxScaled = pd.DataFrame(BUS_102_UP_minMaxScaled, index = BUS_102_UP.iloc[:,:19].index, columns = BUS_102_UP.iloc[:,:19].columns)
# BUS_102_UP_concat = pd.concat([BUS_102_UP_minMaxScaled, BUS_102_UP.iloc[:,19:]], axis = 1)

In [11]:
# import pandas as pd
# import folium 
# from folium import plugins

# time_series = ['6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']

# heat_data = [[[row['GPS_LATI'], row['GPS_LONG'], row[time]] for index, row in BUS_102_UP_concat.iterrows()] for time in time_series]

# m = folium.Map(location=[36.3504119, 127.3845475], tiles='cartodbdark_matter', zoom_start=13)
# heat_map = plugins.HeatMapWithTime(heat_data, index = [time for time in time_series], auto_play = False, radius = 20, max_opacity=1, gradient = {0.2: '#FBD973', 
#                             0.4: '#fa782f', 
#                             0.75: '#F16578', 
#                             1: '#782890'})
# heat_map.add_to(m)
# # m.save('BUS_102_UP.html')

In [12]:
# import folium
# from folium.plugins import HeatMap

# # 주어진 데이터프레임
# data = BUS_102_UP_5

# # 데이터프레임의 GPS_LONG, GPS_LATI, 5 컬럼 사용
# locations = data[['GPS_LATI', 'GPS_LONG', '5']].values.tolist()

# # 지도 초기 설정
# m = folium.Map(location=[36.3504119, 127.3845475], zoom_start=11)

# # 히트맵 레이어 추가
# heatmap_layer = HeatMap(locations, radius=20)
# heatmap_layer.add_to(m)

# # 히트맵 위에 정보를 포함한 Popup 추가
# for location in locations:
#     lat, lon, value = location
#     popup = folium.Popup(f"Value: {value}")
#     folium.Marker([lat, lon], popup=popup).add_to(m)

# # 지도 저장 및 표시
# # m.save('heatmap.html')
# m

In [10]:
import pandas as pd
import folium
from folium import plugins
from sklearn.preprocessing import MinMaxScaler

minMaxScaler = MinMaxScaler()

BUS_NUM_6_TO_22 = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
UP_DOWN_6_TO_22 = ['UP', 'DOWN']
time_series = ['6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']

for bus_num in BUS_NUM_6_TO_22:
    for up_down in UP_DOWN_6_TO_22:
        # min-max 스케일링
        bus_num_str = str(bus_num).zfill(3) # 3자리 숫자로 맞춰줌
        data_to_scale = eval(f'BUS_{bus_num_str}_{up_down}.iloc[:,:19]')  # Extract the data for scaling
        scaled_data = minMaxScaler.fit_transform(data_to_scale)  # Fit and transform the scaler
        BUS_minMaxScaled = pd.DataFrame(scaled_data, index=data_to_scale.index, columns=data_to_scale.columns)
        BUS_concat = pd.concat([BUS_minMaxScaled, eval(f'BUS_{bus_num_str}_{up_down}.iloc[:, 19:]')], axis=1)
        
        # 히트맵 데이터 생성
        heat_data = [[[row['GPS_LATI'], row['GPS_LONG'], row[time]] for index, row in BUS_concat.iterrows()] for time in time_series]
        
        # 히트맵 folium 지도 생성 및 저장
        m = folium.Map(location=[36.3504119, 127.3845475], tiles='cartodbdark_matter', zoom_start=13)
        heat_map = plugins.HeatMapWithTime(heat_data, index=[time for time in time_series], auto_play=False, radius=20, max_opacity=1, gradient={0.1: 'blue', 0.3: 'lime', 0.6: 'yellow', 1: 'red'})
        heat_map.add_to(m)
        m.save(f'./HEATMAP_HTML_FILE/BUS_{bus_num_str}_{up_down}.html')
        # await map_to_png(f'./HEATMAP_CAPTURE/BUS_{bus_num_str}_{up_down}', m) # 출처 : https://yeomss.tistory.com/276


---

BUS_SPEED_DATA

In [11]:
import pandas as pd

merged_df = pd.read_csv('./merged_df.csv', encoding = 'cp949')

In [12]:
merged_df.dropna(axis = 0, inplace = True) # 결측치 제거
merged_df['일자'] = pd.to_datetime(merged_df['일자'], format='%Y-%m-%d') # '일자' 열을 datetime 형식으로 변환
merged_df['DAY'] = merged_df['일자'].dt.day # 'DAY' 열 생성
merged_df = merged_df[merged_df['DAY'].isin([1, 2, 5, 7, 8, 9, 12, 13, 14, 15, 16, 20, 21, 22, 23, 26, 27, 28, 29, 30])] # 평일만 필터링
merged_df['노선'] = merged_df['노선'].astype(int) # 노선 float -> int
merged_df.drop(labels = ['DAY','일자'], axis = 1, inplace = True) # 사용한 행 제거
merged_df.head()

,노선,정류장구간,5:00,6:00,7:00,8:00,9:00,10:00,11:00,12:00,13:00,14:00,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00,평균값
0,102,수통골삼거리-41670▶한밭대학교-41690,24.0,28.0,17.5,16.0,18.8,19.5,19.3,18.8,18.8,21.3,21.8,19.5,18.0,17.3,15.0,21.0,23.0,20.3,17.0,19.7
1,102,한밭대학교-41690▶삼성화재연수원-41730,25.3,30.8,31.0,27.5,27.8,26.5,21.3,21.5,23.8,21.3,21.5,19.8,20.3,19.8,25.5,22.5,23.0,20.5,21.0,23.7
2,102,삼성화재연수원-41730▶하우스토리/신협연수원-41740,26.0,30.5,25.8,23.0,23.5,23.0,24.3,25.0,22.0,25.5,26.3,25.3,24.8,21.0,31.3,25.5,22.0,24.3,21.8,24.8
3,102,하우스토리/신협연수원-41740▶덕명네거리-41400,22.0,26.8,21.5,22.0,22.5,23.5,26.5,25.3,25.3,24.0,27.0,26.0,21.8,21.3,24.3,26.0,27.5,30.8,24.3,24.7
4,102,덕명네거리-41400▶주유소-41420,24.3,33.3,32.3,27.5,30.3,29.5,28.8,33.8,29.8,31.8,28.0,27.8,25.3,29.3,33.0,33.8,31.0,31.0,28.0,29.9


In [16]:
def bus_speed(merged_df, route_num):
    exdata = merged_df[merged_df['노선']==route_num]
    exdata_mean = exdata.groupby('정류장구간').mean()
    exdata_mean.drop(labels = '노선', axis = 1, inplace = True)
    exdata_mean.reset_index(inplace = True)
    exdata_mean['정류장구간_1'] = exdata_mean["정류장구간"].apply(lambda x: x.split('▶')[0].split('-')[1]).astype(int)
    exdata_mean["정류장구간_2"] = exdata_mean["정류장구간"].apply(lambda x: x.split('▶')[1].split('-')[1]).astype(int)
    exdata_mean.drop(labels = ['정류장구간'], axis = 1, inplace = True)
    exdata_mean = exdata_mean[['정류장구간_1', '정류장구간_2','5:00', '6:00', '7:00', '8:00', '9:00', '10:00', '11:00', '12:00', '13:00',
                               '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']]

    return exdata_mean

In [27]:
# 버스 별 평균 SPEED 데이터프레임 생성 코드

bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802] # 위에서 str타입으로 불러왔으니 다시 불러오기

for bus_number in bus_numbers:
    variable_name = f"BUS_SPEED_{bus_number}_MEAN" # BUS_SPEED_102_MEAN
    globals()[variable_name] = bus_speed(merged_df, bus_number)

In [23]:
# BUS_SPEED_102 = merged_df[merged_df['노선']==102]
# BUS_SPEED_102_MEAN = BUS_SPEED_102.groupby('정류장구간').mean()
# BUS_SPEED_102_MEAN.drop(labels = '노선', axis = 1, inplace = True)
# BUS_SPEED_102_MEAN.reset_index(inplace = True)
# BUS_SPEED_102_MEAN["정류장구간_1"] = BUS_SPEED_102_MEAN["정류장구간"].apply(lambda x: x.split('▶')[0].split('-')[1]).astype(int)
# BUS_SPEED_102_MEAN["정류장구간_2"] = BUS_SPEED_102_MEAN["정류장구간"].apply(lambda x: x.split('▶')[1].split('-')[1]).astype(int)
# BUS_SPEED_102_MEAN.drop(labels = ['정류장구간'], axis = 1, inplace = True)
# BUS_SPEED_102_MEAN = BUS_SPEED_102_MEAN[['정류장구간_1', '정류장구간_2','5:00', '6:00', '7:00', '8:00', '9:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', 
#                                          '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']]
# BUS_SPEED_102_MEAN

---

In [29]:
def get_bus_stop_tp(getstationbyrouteall_df, ROUTE_BUS_NUMBER, ROUTE_NO):
    # 주어진 버스 노선 번호에 해당하는 버스 노선 코드를 얻은 뒤, 노선 코드에 해당하는 버스 정류장 데이터 추출 <순서대로 진행하지 않으면 버스 정류장 데이터가 중복 됨>
    exdata = getstationbyrouteall_df[getstationbyrouteall_df['ROUTE_CD']==ROUTE_BUS_NUMBER[ROUTE_BUS_NUMBER['ROUTE_NO']==ROUTE_NO]['ROUTE_CD'].values[0]].reset_index(drop=True)
    
    return exdata

In [28]:
# # 필요한 노선만 필터링한 뒤, int형으로 변환
# bus_numbers = ['102', '103', '105', '106', '108', '115', '119', '201', '211', '301', '311', '314', '511', '603', '604', '605', '613', '615', '703', '704', '705', '706', '802']
# ROUTE_BUS_NUMBER = getrouteinfoall_df[getrouteinfoall_df['ROUTE_NO'].isin(bus_numbers)]
# ROUTE_BUS_NUMBER['ROUTE_NO'] = ROUTE_BUS_NUMBER['ROUTE_NO'].astype(int)

In [37]:
# BUS_SPEED_ROUTE_102 = get_bus_stop_tp(getstationbyrouteall_df2, ROUTE_BUS_NUMBER, 102)
# BUS_SPEED_ROUTE_102

In [31]:
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802] # 위에서 str타입으로 불러왔으니 다시 불러오기

for bus_number in bus_numbers:
    variable_name = f"BUS_SPEED_ROUTE_{bus_number}" # BUS_SPEED_ROUTE_102
    globals()[variable_name] = get_bus_stop_tp(getstationbyrouteall_df2, ROUTE_BUS_NUMBER, bus_number)

In [38]:
from shapely.geometry import Point, LineString
import geopandas as gpd

def make_coordinate(bus_speed_mean_data, bus_speed_route_data):
    exdata_merge = bus_speed_mean_data.merge(bus_speed_route_data, left_on = '정류장구간_1', right_on = 'BUS_STOP_ID')
    exdata_merge.rename(columns = {'BUSSTOP_SEQ' : 'STATION_1_BUSSTOP_SEQ', 'BUSSTOP_TP' : 'STATION_1_BUSSTOP_TP', 'GPS_LATI' : 'STATION_1_GPS_LATI', 'GPS_LONG' : 'STATION_1_GPS_LONG'}, inplace = True)
    exdata_merge = exdata_merge[['정류장구간_1','STATION_1_BUSSTOP_SEQ','STATION_1_BUSSTOP_TP', 'STATION_1_GPS_LATI','STATION_1_GPS_LONG', 
                                                        '정류장구간_2', '5:00', '6:00', '7:00', '8:00', '9:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', 
                                                        '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']]
    exdata_merge = exdata_merge.merge(bus_speed_route_data, left_on = '정류장구간_2', right_on = 'BUS_STOP_ID')
    exdata_merge.rename(columns = {'BUSSTOP_SEQ' : 'STATION_2_BUSSTOP_SEQ', 'BUSSTOP_TP' : 'STATION_2_BUSSTOP_TP', 'GPS_LATI' : 'STATION_2_GPS_LATI', 'GPS_LONG' : 'STATION_2_GPS_LONG'}, inplace = True)
    exdata_merge = exdata_merge[['정류장구간_1','STATION_1_BUSSTOP_SEQ','STATION_1_BUSSTOP_TP', 'STATION_1_GPS_LATI','STATION_1_GPS_LONG', 
                                                        '정류장구간_2','STATION_2_BUSSTOP_SEQ','STATION_2_BUSSTOP_TP', 'STATION_2_GPS_LATI','STATION_2_GPS_LONG',
                                                        '5:00', '6:00', '7:00', '8:00', '9:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']]
    
    bus_speed_mean_coordinate_data = exdata_merge.copy()
    
    # 지오메트리 컬럼 생성
    bus_speed_mean_coordinate_data["STATION_1_GEOMETRY"] = bus_speed_mean_coordinate_data.apply(lambda row: Point(row["STATION_1_GPS_LATI"], row["STATION_1_GPS_LONG"]), axis=1)
    bus_speed_mean_coordinate_data["STATION_2_GEOMETRY"] = bus_speed_mean_coordinate_data.apply(lambda row: Point(row["STATION_2_GPS_LATI"], row["STATION_2_GPS_LONG"]), axis=1)
    bus_speed_mean_coordinate_data.drop(labels = ['STATION_1_GPS_LATI','STATION_1_GPS_LONG','STATION_2_GPS_LATI','STATION_2_GPS_LONG'], axis = 1, inplace = True)
    
    # point를 linestring으로 변환해주기
    bus_speed_mean_coordinate_data["LineString"] = [LineString([row["STATION_1_GEOMETRY"], row["STATION_2_GEOMETRY"]]).wkt for _, row in bus_speed_mean_coordinate_data.iterrows()]
    bus_speed_mean_coordinate_data.drop(labels = ['STATION_1_GEOMETRY','STATION_2_GEOMETRY','정류장구간_1','정류장구간_2'], axis = 1, inplace = True)

    # SECTION 열 생성
    bus_speed_mean_coordinate_data["SECTION"] = bus_speed_mean_coordinate_data["STATION_1_BUSSTOP_SEQ"].astype(str) + "~" + bus_speed_mean_coordinate_data["STATION_2_BUSSTOP_SEQ"].astype(str)
    bus_speed_mean_coordinate_data.drop(labels = ['STATION_1_BUSSTOP_SEQ','STATION_2_BUSSTOP_SEQ'], axis = 1, inplace = True)
    bus_speed_mean_coordinate_data = bus_speed_mean_coordinate_data[['SECTION','LineString','6:00', '7:00', '8:00', '9:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00',
                                                                '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00','STATION_1_BUSSTOP_TP', 'STATION_2_BUSSTOP_TP']]
    return bus_speed_mean_coordinate_data

In [20]:
# BUS_SPEED_102_MEAN_MERGE = BUS_SPEED_102_MEAN.merge(BUS_SPEED_ROUTE_102, left_on = '정류장구간_1', right_on = 'BUS_STOP_ID')
# BUS_SPEED_102_MEAN_MERGE.rename(columns = {'BUSSTOP_SEQ' : 'STATION_1_BUSSTOP_SEQ', 'BUSSTOP_TP' : 'STATION_1_BUSSTOP_TP', 'GPS_LATI' : 'STATION_1_GPS_LATI', 'GPS_LONG' : 'STATION_1_GPS_LONG'}, inplace = True)
# BUS_SPEED_102_MEAN_MERGE = BUS_SPEED_102_MEAN_MERGE[['정류장구간_1','STATION_1_BUSSTOP_SEQ','STATION_1_BUSSTOP_TP', 'STATION_1_GPS_LATI','STATION_1_GPS_LONG', 
#                                                      '정류장구간_2', '5:00', '6:00', '7:00', '8:00', '9:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', 
#                                                      '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']]
# BUS_SPEED_102_MEAN_MERGE = BUS_SPEED_102_MEAN_MERGE.merge(BUS_SPEED_ROUTE_102, left_on = '정류장구간_2', right_on = 'BUS_STOP_ID')
# BUS_SPEED_102_MEAN_MERGE.rename(columns = {'BUSSTOP_SEQ' : 'STATION_2_BUSSTOP_SEQ', 'BUSSTOP_TP' : 'STATION_2_BUSSTOP_TP', 'GPS_LATI' : 'STATION_2_GPS_LATI', 'GPS_LONG' : 'STATION_2_GPS_LONG'}, inplace = True)
# BUS_SPEED_102_MEAN_MERGE = BUS_SPEED_102_MEAN_MERGE[['정류장구간_1','STATION_1_BUSSTOP_SEQ','STATION_1_BUSSTOP_TP', 'STATION_1_GPS_LATI','STATION_1_GPS_LONG', 
#                                                      '정류장구간_2','STATION_2_BUSSTOP_SEQ','STATION_2_BUSSTOP_TP', 'STATION_2_GPS_LATI','STATION_2_GPS_LONG',
#                                                      '5:00', '6:00', '7:00', '8:00', '9:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00']]
# BUS_SPEED_102_MEAN_MERGE.head(2)

,정류장구간_1,STATION_1_BUSSTOP_SEQ,STATION_1_BUSSTOP_TP,STATION_1_GPS_LATI,STATION_1_GPS_LONG,정류장구간_2,STATION_2_BUSSTOP_SEQ,STATION_2_BUSSTOP_TP,STATION_2_GPS_LATI,STATION_2_GPS_LONG,5:00,6:00,7:00,8:00,9:00,10:00,11:00,12:00,13:00,14:00,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00
0,32650,23,,36.357365,127.395950,50040,24,,36.357296,127.407845,26.015,23.795,19.18,14.385,17.56,17.320,17.310,18.865,19.415,18.715,17.66,17.015,15.045,15.475,18.845,19.325,21.50,23.72,24.325
1,11540,37,,36.346535,127.444756,11560,38,,36.344350,127.446490,20.855,19.610,17.27,16.085,16.26,16.415,16.745,16.810,16.710,16.985,16.29,15.780,15.830,14.460,15.050,16.080,17.26,19.39,16.665


In [21]:
from shapely.geometry import Point, LineString
import geopandas as gpd

BUS_SPEED_102_MEAN_COORDINATE = BUS_SPEED_102_MEAN_MERGE.copy()

# 지오메트리 컬럼 생성
BUS_SPEED_102_MEAN_COORDINATE["STATION_1_GEOMETRY"] = BUS_SPEED_102_MEAN_COORDINATE.apply(lambda row: Point(row["STATION_1_GPS_LATI"], row["STATION_1_GPS_LONG"]), axis=1)
BUS_SPEED_102_MEAN_COORDINATE["STATION_2_GEOMETRY"] = BUS_SPEED_102_MEAN_COORDINATE.apply(lambda row: Point(row["STATION_2_GPS_LATI"], row["STATION_2_GPS_LONG"]), axis=1)
BUS_SPEED_102_MEAN_COORDINATE.drop(labels = ['STATION_1_GPS_LATI','STATION_1_GPS_LONG','STATION_2_GPS_LATI','STATION_2_GPS_LONG'], axis = 1, inplace = True)
BUS_SPEED_102_MEAN_COORDINATE.head(2)

,정류장구간_1,STATION_1_BUSSTOP_SEQ,STATION_1_BUSSTOP_TP,정류장구간_2,STATION_2_BUSSTOP_SEQ,STATION_2_BUSSTOP_TP,5:00,6:00,7:00,8:00,9:00,10:00,11:00,12:00,13:00,14:00,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00,STATION_1_GEOMETRY,STATION_2_GEOMETRY
0,32650,23,,50040,24,,26.015,23.795,19.18,14.385,17.56,17.320,17.310,18.865,19.415,18.715,17.66,17.015,15.045,15.475,18.845,19.325,21.50,23.72,24.325,POINT (36.357365 127.39595),POINT (36.357296 127.407845)
1,11540,37,,11560,38,,20.855,19.610,17.27,16.085,16.26,16.415,16.745,16.810,16.710,16.985,16.29,15.780,15.830,14.460,15.050,16.080,17.26,19.39,16.665,POINT (36.346535 127.444756),POINT (36.34435 127.44649)


In [22]:
# point를 linestring으로 변환해주기
BUS_SPEED_102_MEAN_COORDINATE["LineString"] = [LineString([row["STATION_1_GEOMETRY"], row["STATION_2_GEOMETRY"]]).wkt for _, row in BUS_SPEED_102_MEAN_COORDINATE.iterrows()]
BUS_SPEED_102_MEAN_COORDINATE.drop(labels = ['STATION_1_GEOMETRY','STATION_2_GEOMETRY','정류장구간_1','정류장구간_2'], axis = 1, inplace = True)

# SECTION 열 생성
BUS_SPEED_102_MEAN_COORDINATE["SECTION"] = BUS_SPEED_102_MEAN_COORDINATE["STATION_1_BUSSTOP_SEQ"].astype(str) + "~" + BUS_SPEED_102_MEAN_COORDINATE["STATION_2_BUSSTOP_SEQ"].astype(str)
BUS_SPEED_102_MEAN_COORDINATE.drop(labels = ['STATION_1_BUSSTOP_SEQ','STATION_2_BUSSTOP_SEQ'], axis = 1, inplace = True)
BUS_SPEED_102_MEAN_COORDINATE = BUS_SPEED_102_MEAN_COORDINATE[['SECTION','LineString','6:00', '7:00', '8:00', '9:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00',
                                                               '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00','STATION_1_BUSSTOP_TP', 'STATION_2_BUSSTOP_TP']]
BUS_SPEED_102_MEAN_COORDINATE.head(2)

,SECTION,LineString,6:00,7:00,8:00,9:00,10:00,11:00,12:00,13:00,14:00,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,STATION_1_BUSSTOP_TP,STATION_2_BUSSTOP_TP
0,23~24,"LINESTRING (36.357365 127.39595, 36.357296 127...",23.795,19.18,14.385,17.56,17.320,17.310,18.865,19.415,18.715,17.66,17.015,15.045,15.475,18.845,19.325,21.50,23.72,,
1,37~38,"LINESTRING (36.346535 127.444756, 36.34435 127...",19.610,17.27,16.085,16.26,16.415,16.745,16.810,16.710,16.985,16.29,15.780,15.830,14.460,15.050,16.080,17.26,19.39,,


In [23]:
# speed range행 생성
BUS_SPEED_102_MEAN_COORDINATE_10 = BUS_SPEED_102_MEAN_COORDINATE[['SECTION','LineString','10:00','STATION_1_BUSSTOP_TP','STATION_2_BUSSTOP_TP']]
BUS_SPEED_102_MEAN_COORDINATE_10.loc[BUS_SPEED_102_MEAN_COORDINATE_10['10:00'] >= 20, 'speed_range'] = 'fast'
BUS_SPEED_102_MEAN_COORDINATE_10.loc[(BUS_SPEED_102_MEAN_COORDINATE_10['10:00'] >= 10) & (BUS_SPEED_102_MEAN_COORDINATE_10['10:00'] < 20), 'speed_range'] = 'medium'
BUS_SPEED_102_MEAN_COORDINATE_10.loc[BUS_SPEED_102_MEAN_COORDINATE_10['10:00'] < 10, 'speed_range'] = 'slow'
BUS_SPEED_102_MEAN_COORDINATE_10.sort_values('SECTION')

,SECTION,LineString,10:00,STATION_1_BUSSTOP_TP,STATION_2_BUSSTOP_TP,speed_range
76,10~11,"LINESTRING (36.359417 127.325035, 36.35619 127...",19.110,,,medium
10,11~12,"LINESTRING (36.35619 127.33231, 36.355476 127....",14.995,,,medium
30,12~13,"LINESTRING (36.355476 127.334816, 36.35465 127...",14.880,,,medium
31,13~14,"LINESTRING (36.35465 127.33812, 36.354378 127....",15.950,,,medium
60,14~15,"LINESTRING (36.354378 127.34202, 36.35811 127....",15.850,,,medium
...,...,...,...,...,...,...
82,89~90,"LINESTRING (36.35188 127.29814, 36.348263 127....",18.255,,,medium
8,8~9,"LINESTRING (36.35907 127.31484, 36.35935 127.3...",27.275,,,fast
36,90~1,"LINESTRING (36.348263 127.29597, 36.346115 127...",32.235,,1,fast
37,90~91,"LINESTRING (36.348263 127.29597, 36.346436 127...",32.235,,3,fast


In [24]:
# 자릿수 맞춰주기 ex) 1~2 -> 01~02 이렇게 해야 sort_values()했을 때 순서대로 정렬됨
def format_section(section):
    parts = section.split('~')
    formatted_parts = [part.zfill(2) for part in parts]
    return f"{formatted_parts[0]}~{formatted_parts[1]}"

# 적용
BUS_SPEED_102_MEAN_COORDINATE_10['SECTION'] = BUS_SPEED_102_MEAN_COORDINATE_10['SECTION'].apply(format_section)
BUS_SPEED_102_MEAN_COORDINATE_10.sort_values('SECTION',inplace = True)

In [54]:
import folium
from shapely.wkt import loads
import pandas as pd

m = folium.Map(location=[36.365, 127.352], tiles='cartodbdark_matter', zoom_start=12)

# speed range 색상 지정해주기
colormap = {
    'slow': 'red',
    'medium': 'green',
    'fast': 'blue'
}

# Linestring 지도에 표시하는 코드
for idx, row in BUS_SPEED_102_MEAN_COORDINATE_10[40:].iterrows():
    if row['LineString'] is not None:
        line = loads(row['LineString'])
        coords = line.coords
        color = colormap.get(row['speed_range'], 'gray') # speed_range에 따른 색상 설정, 미지정 시 회색 사용
        folium.PolyLine(locations=coords, color=color, weight=7, popup=f"Speed: {row['10:00']} km/h").add_to(m) # PolyLine 객체를 생성하여 지도에 추가하고 팝업으로 속도 정보를 표시
        
# 범례 (legend) 추가해주는 코드
legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; width: 100px; height: 100px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; opacity: 0.8;">
     <p style="margin: 5px;"><span style="color: red;">&#9679;</span> Slow</p>
     <p style="margin: 5px;"><span style="color: green;">&#9679;</span> Medium</p>
     <p style="margin: 5px;"><span style="color: blue;">&#9679;</span> Fast</p>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))
m

In [63]:
# m.save('bus_102_10.html')

In [64]:
from folium import utilities
from pyppeteer import launch

# 만든 지도를 png 파일로 캡쳐해서 저장하는 함수
async def map_to_png(target, m): # target은 저장할 파일 이름
    html = m.get_root().render()
    browser = await launch(headless=True)

    page = await browser.newPage()
    with utilities.temp_html_filepath(html) as fname:
        await page.goto('file://{path}'.format(path=fname))

    img_data = await page.screenshot({'path': f'{target}.png', 'fullPage': 'true', })
    await browser.close()

In [60]:
await map_to_png('bus_102_10', m) # 출처 : https://yeomss.tistory.com/276